In [1]:
import sys
sys.path

['C:\\Users\\Ravi Varma Injeti',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\python37.zip',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\DLLs',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3',
 '',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib\\site-packages',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Ravi Varma Injeti\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Ravi Varma Injeti\\.ipython']

In [2]:
import pandas as pd

file_path = "E:/Topic Modelling/"
raw_data = pd.read_csv(file_path + 'abcnews-date-text.csv')
raw_data.columns


raw_data_text = raw_data[:300000][['headline_text']];
raw_data_text['index'] =raw_data_text.index
documents = raw_data_text

print(len(documents))


300000


In [3]:

documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [4]:
#!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to C:\Users\Ravi Varma
[nltk_data]     Injeti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v'))

go


In [6]:
##function for lemarizing a each test using wornet lematizer
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos = 'v'))
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocessing_text(text):
    results = []
    for tokens in gensim.utils.simple_preprocess(text):
        if tokens not in gensim.parsing.preprocessing.STOPWORDS and len(tokens)>3:
            stem = lemmatize_stemming(tokens)
            results.append(stem)
    return results
            
    

In [7]:
document_num = 402
raw_text = documents[documents['index'] == document_num].values[0][0]

print("raw_text: ", raw_text)

words=[]
for word in raw_text.split():
    words.append(word)

print(words)    
print("pre_processed_text: ", preprocessing_text(raw_text))

raw_text:  seaman injury crisis for arsenal
['seaman', 'injury', 'crisis', 'for', 'arsenal']
pre_processed_text:  ['seaman', 'injuri', 'crisi', 'arsenal']


In [8]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
299995,commuters evacuated from circular quay train,299995
299996,companies invest billions in png gas industry,299996
299997,company looks to expand basalt quarry,299997
299998,cooloola mayor flags support for super council,299998


In [9]:
#preprocess all the headlines, saving the list of results as 'processed_docs'

preprocessed_docs = documents['headline_text'].map(preprocessing_text)  

preprocessed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [10]:
#creating a dictionary using the gensim corpora dictionary for the preprocessed documents.
dictionary = gensim.corpora.Dictionary(preprocessed_docs)

count = 0
for words, tags in dictionary.iteritems():
    print(words, tags)
    count = count + 1
    if count > 50:
        break;
    

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
11 aust
12 rise
13 staff
14 strike
15 affect
16 australian
17 travel
18 ambiti
19 jump
20 olsson
21 tripl
22 win
23 antic
24 barca
25 break
26 delight
27 record
28 aussi
29 match
30 memphi
31 qualifi
32 stosur
33 wast
34 address
35 council
36 iraq
37 secur
38 australia
39 lock
40 timet
41 contribut
42 million
43 birthday
44 celebr
45 robson
46 ahead
47 bathhous
48 plan
49 championship
50 cycl


In [11]:
###creating the BAG_of_word model dictionary to find the how man words and how many times a word appear.

doc_corpus=[dictionary.doc2bow(doc) for doc in preprocessed_docs]

doc_corpus[document_num]

[(449, 1), (630, 1), (864, 1), (1074, 1)]

In [12]:
###creating a tf-idf vectorizer and initializing the model
from gensim import corpora, models

tfidf_model = models.TfidfModel(doc_corpus) 

##transforming the tf_idf models
tfidf= tfidf_model[doc_corpus]
print(tfidf[:5])

In [13]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in tfidf:
    pprint(doc)
    break

[(0, 0.5959919082495837),
 (1, 0.3920069955308767),
 (2, 0.48532280284497653),
 (3, 0.5055550788930631)]


In [15]:
model = gensim.models.LdaModel(doc_corpus, 
                                    num_topics = 10, 
                                    id2word = dictionary,                                    
                                    passes = 50)


In [18]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.024*"help" + 0.019*"drought" + 0.018*"work" + 0.018*"govt" + 0.017*"farmer" + 0.016*"resid" + 0.015*"indigen" + 0.014*"offer" + 0.014*"return" + 0.013*"urg" 
Words: 0


Topic: 0.026*"claim" + 0.023*"jail" + 0.021*"reject" + 0.016*"australian" + 0.015*"year" + 0.015*"aust" + 0.014*"market" + 0.013*"fear" + 0.012*"high" + 0.011*"timor" 
Words: 1


Topic: 0.026*"open" + 0.022*"back" + 0.020*"world" + 0.019*"record" + 0.016*"teen" + 0.015*"expect" + 0.015*"play" + 0.014*"develop" + 0.014*"final" + 0.013*"look" 
Words: 2


Topic: 0.048*"govt" + 0.018*"hospit" + 0.017*"chang" + 0.017*"health" + 0.017*"school" + 0.017*"call" + 0.016*"say" + 0.016*"servic" + 0.016*"minist" + 0.015*"fund" 
Words: 3


Topic: 0.069*"polic" + 0.024*"crash" + 0.023*"investig" + 0.018*"road" + 0.017*"test" + 0.016*"elect" + 0.014*"driver" + 0.013*"mayor" + 0.013*"probe" + 0.011*"assault" 
Words: 4


Topic: 0.051*"water" + 0.037*"council" + 0.028*"plan" + 0.020*"continu" + 0.020*"push" + 0.018*"nation" + 0.0

In [21]:
###Running LDA model uisng tfidf model

tf_idf_model = gensim.models.LdaModel(tfidf, 
                                       id2word = dictionary,
                                       num_topics = 10,
                                       passes = 50)

In [22]:
for idx, topic in tf_idf_model.print_topics(-1):
    print("Topic : {} \nWords: {} ".format(idx, topic))
    print("\n")

Topic : 0 
Words: 0.015*"teen" + 0.012*"crew" + 0.011*"prompt" + 0.011*"open" + 0.011*"green" + 0.009*"research" + 0.008*"station" + 0.008*"remain" + 0.008*"futur" + 0.007*"deni" 


Topic : 1 
Words: 0.014*"rudd" + 0.014*"troop" + 0.011*"leader" + 0.010*"market" + 0.010*"fire" + 0.009*"aust" + 0.009*"iraq" + 0.009*"nuclear" + 0.009*"solomon" + 0.008*"north" 


Topic : 2 
Words: 0.011*"stand" + 0.011*"arrest" + 0.010*"climat" + 0.010*"sale" + 0.009*"bush" + 0.009*"condit" + 0.009*"qanta" + 0.008*"trade" + 0.008*"make" + 0.008*"howard" 


Topic : 3 
Words: 0.011*"rescu" + 0.010*"rate" + 0.010*"train" + 0.009*"debat" + 0.009*"propos" + 0.008*"soldier" + 0.008*"central" + 0.008*"pledg" + 0.008*"fin" + 0.008*"japan" 


Topic : 4 
Words: 0.018*"water" + 0.014*"govt" + 0.011*"plan" + 0.011*"hick" + 0.010*"council" + 0.010*"break" + 0.009*"hous" + 0.008*"fund" + 0.008*"communiti" + 0.008*"sydney" 


Topic : 5 
Words: 0.024*"crash" + 0.023*"polic" + 0.020*"charg" + 0.019*"investig" + 0.014*"kil

In [24]:
preprocessed_docs[600]

['cost', 'blowout', 'forecast', 'water', 'treatment', 'plan']

In [31]:
document_num = 4310
# Our test document is document number 600


# Our test document is document number 600
for index, score in sorted(model[doc_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))


Score: 0.5273331999778748	 
Topic: 0.024*"help" + 0.019*"drought" + 0.018*"work" + 0.018*"govt" + 0.017*"farmer" + 0.016*"resid" + 0.015*"indigen" + 0.014*"offer" + 0.014*"return" + 0.013*"urg"

Score: 0.37264299392700195	 
Topic: 0.051*"water" + 0.037*"council" + 0.028*"plan" + 0.020*"continu" + 0.020*"push" + 0.018*"nation" + 0.017*"miss" + 0.015*"boost" + 0.015*"consid" + 0.015*"meet"

Score: 0.012507086619734764	 
Topic: 0.048*"govt" + 0.018*"hospit" + 0.017*"chang" + 0.017*"health" + 0.017*"school" + 0.017*"call" + 0.016*"say" + 0.016*"servic" + 0.016*"minist" + 0.015*"fund"

Score: 0.012503442354500294	 
Topic: 0.040*"urg" + 0.023*"hous" + 0.023*"labor" + 0.020*"worker" + 0.019*"death" + 0.019*"govt" + 0.018*"polic" + 0.016*"industri" + 0.016*"arrest" + 0.015*"public"

Score: 0.012502958066761494	 
Topic: 0.031*"say" + 0.025*"opposit" + 0.024*"forc" + 0.024*"power" + 0.017*"take" + 0.015*"time" + 0.015*"lead" + 0.015*"leader" + 0.013*"storm" + 0.013*"prompt"

Score: 0.0125023545

In [32]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(tf_idf_model[doc_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, tf_idf_model.print_topic(index, 10)))


Score: 0.887478768825531	 
Topic: 0.014*"blaze" + 0.011*"firefight" + 0.010*"flood" + 0.009*"water" + 0.009*"campaign" + 0.009*"local" + 0.009*"east" + 0.008*"timor" + 0.008*"poll" + 0.008*"control"

Score: 0.012503419071435928	 
Topic: 0.018*"water" + 0.014*"govt" + 0.011*"plan" + 0.011*"hick" + 0.010*"council" + 0.010*"break" + 0.009*"hous" + 0.008*"fund" + 0.008*"communiti" + 0.008*"sydney"

Score: 0.012503261677920818	 
Topic: 0.014*"rudd" + 0.014*"troop" + 0.011*"leader" + 0.010*"market" + 0.010*"fire" + 0.009*"aust" + 0.009*"iraq" + 0.009*"nuclear" + 0.009*"solomon" + 0.008*"north"

Score: 0.012502910569310188	 
Topic: 0.011*"stand" + 0.011*"arrest" + 0.010*"climat" + 0.010*"sale" + 0.009*"bush" + 0.009*"condit" + 0.009*"qanta" + 0.008*"trade" + 0.008*"make" + 0.008*"howard"

Score: 0.01250283233821392	 
Topic: 0.038*"closer" + 0.012*"miss" + 0.011*"search" + 0.011*"resid" + 0.009*"worker" + 0.009*"drought" + 0.009*"bushfir" + 0.008*"farm" + 0.008*"author" + 0.008*"rain"

Score:

In [35]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocessing_text(unseen_document))
print(bow_vector)

for index, score in sorted(model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, model.print_topic(index, 5)))

[(399, 1), (1086, 1), (1832, 1), (3015, 1), (8850, 1)]
Score: 0.6200794577598572	 Topic: 0.044*"charg" + 0.037*"court" + 0.035*"face" + 0.024*"closer" + 0.024*"accus"
Score: 0.2199477255344391	 Topic: 0.048*"govt" + 0.018*"hospit" + 0.017*"chang" + 0.017*"health" + 0.017*"school"
Score: 0.019996600225567818	 Topic: 0.024*"help" + 0.019*"drought" + 0.018*"work" + 0.018*"govt" + 0.017*"farmer"
Score: 0.019996600225567818	 Topic: 0.026*"claim" + 0.023*"jail" + 0.021*"reject" + 0.016*"australian" + 0.015*"year"
Score: 0.019996600225567818	 Topic: 0.026*"open" + 0.022*"back" + 0.020*"world" + 0.019*"record" + 0.016*"teen"
Score: 0.019996600225567818	 Topic: 0.069*"polic" + 0.024*"crash" + 0.023*"investig" + 0.018*"road" + 0.017*"test"
Score: 0.019996600225567818	 Topic: 0.051*"water" + 0.037*"council" + 0.028*"plan" + 0.020*"continu" + 0.020*"push"
Score: 0.019996600225567818	 Topic: 0.031*"say" + 0.025*"opposit" + 0.024*"forc" + 0.024*"power" + 0.017*"take"
Score: 0.019996600225567818	 Top